In [0]:
%run /4-37am/AB_tests_framework/ab_tests_related_functions

Input configurations for the experiment calculations

In [0]:
# inputs:

experiment_name = 'Test_of_new_feature_for_notebook_2' # Name of experiment. Try to make it unique
test_start_date = '2022-06-10' # First date of roll-out of the experiment
calc_start_date = '2022-06-12' # First date of results calculation. Results well be calculate from test_start_date up to date between calc_start_date and calc_end_date.
calc_end_date = '2022-07-17' #If you need to calculate results just for 1 day make calc_start_date=calc_end_date
# yesterday=(pd.to_datetime("today") - pd.Timedelta(1, unit='D')).strftime('%Y-%m-%d')
calc_frequency = '7d' # frequency of calculations between calc_start_date and calc_end_date. Example: if calc_frequency='7d' and calc_start_date ='2022-01-01' and calc_end_date='2022-01-22' then calculations will be performed on: 2022-01-01, 2022-01-08,2022-01-15,2022-01-22. More info on frequency types: https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases 
dates=['2022-07-20', '2022-07-22'] # you can input list of dates to calculate result for each iteratively e.g. for [test_start_date;date_1],[test_start_date,date_2].. if yo want to calculate results daily or by given frequency just leave it empty. Note that number of dates in dates list should be > 2


metrics_columns = ['gmv','items','orders'] # Names of metrics you need to calculate stat.tests. Should be the same as names in sql query results table.

proportions_metrics_flag = True # if you need to calculate proportion metrics in 'numerator/denumerator' format
proportions_columns = {'over100_customers':'all_customers'} # names of metrics from sql query that you want to use for proportion metrics in numerator:denumerator format. Note that values per user must be binary 0 or 1. 

ratio_metrics_flag = True # if you need to calculate ratio metrics in 'numerator/denumerator' format
ratio_columns = {'items':'orders'} # names of metrics from sql query that you want to use for ratio metrics in numerator:denumerator format. 

need_bootstrap = True # True if you want to calculate Bootstrap statistics 
need_boostrap_wo_outliers = True # True if you want to calculate Bootstrap statistics based on data with removed outliers
need_buckets = False # True if you want to calculate ttest based on buckets split
need_buckets_wo_outliers = False # True if you want to calculate ttest based on buckets split of data with removed outliers
need_mde = True # True if you need to calculate MDE based on avaialable data

quantile_for_outliers = 0.99999 # quantile for outliers removing. Users who have metric value above this quentile will be removed for 'without outliers' calculations
bootstrap_iterations = 1000 # number of iterations in bootstrap. You can 
buckets_num = 150 # number of buckets to split (more buckets -> less customers in one bucket -> less precision; less buckets -> less sample for ttest -> less precision, so it is always a tradeoff. Recomendation is not to use less than 60 buckets and have number of customers in bucket not less than 4000)

variant_col = 'payload_variant' #name of column in sql resulted table that contains names of treatment and control groups
control_gr_name = 'Invoice' #name of control group in variant_col
treatment_gr_name = 'BNPL2.0' #name of treatment group in variant_col

In [0]:
%scala
// Input name of table where rusults should be placed (table should be created before the start of calculations. if not - uncomment and run cmd 7)
val TargetTable = "zps_analytics.ab_test_calculations_new_approach"

TargetTable: String = zps_analytics.ab_test_calculations_new_approach

In [0]:
%scala
// Skip this cmd if you want to attach results to existing tableau dashboard
// create table for stat.tests results collection on RedShift if not yet created:

// val TargetTable = "zps_analytics.ab_test_calculations_new_approach"

// dwhSend(s"""
//   -- Want to rebuild the full table? Then uncomment the next line. :)
//   drop table if exists $TargetTable;
//   create table if not exists $TargetTable (
//        experiment_name varchar(255)
//        ,dt date
//        ,metric varchar(255)
//        ,metric_type varchar(255)
//        ,mean_control float
//        ,mean_treatment float
//        ,mean_diff float
//        ,mean_control_wo_outliers float
//        ,mean_treatment_wo_outliers float
//        ,mean_diff_wo_outliers float
//        ,median_control float
//        ,median_treatment float
//        ,median_diff float
//        ,ttest_pval float
//        ,ttest_mean_diff float
//        ,ttest_ci_prc_low float
//        ,ttest_ci_prc_up float
//        ,ttest_wo_outliers_pval float
//        ,ttest_wo_outliers_mean_diff float
//        ,ttest_wo_outliers_ci_prc_low float
//        ,ttest_wo_outliers_ci_prc_up float
//        ,btstrp_pval float
//        ,btstrp_mean_diff float
//        ,btstrp_ci_prc_low float
//        ,btstrp_ci_prc_up float
//        ,btstrp_wo_outliers_pval float
//        ,btstrp_wo_outliers_mean_diff float
//        ,btstrp_wo_outliers_ci_prc_low float
//        ,btstrp_wo_outliers_ci_prc_up float
//        ,buckets_pval float
//        ,buckets_mean_diff float
//        ,buckets_ci_prc_low float
//        ,buckets_ci_prc_up float
//        ,buckets_wo_outliers_pval float
//        ,buckets_wo_outliers_mean_diff float
//        ,buckets_wo_outliers_ci_prc_low float
//        ,buckets_wo_outliers_ci_prc_up float
//        ,MW_pval float
//        ,chi2_pval float                        
//        ,outliers_percentile float
//        ,outliers_threshold float
//        ,calculations_time varchar(255)
//        ,mde_bootstrap float
//        ,mde_ttest_wo_outliers float
//        ,units_control float
//        ,units_treatment float
//        ,srm_pval float
//   );
  
//   -- share with all Redshift users (mark as comment if not wished)
//   grant select on $TargetTable to public
// """)

//  // if you need to add new column:
// val sqlAdd = s"ALTER TABLE $TargetTable ADD COLUMN srm_pval float DEFAULT Null"
// dwhSend(sqlAdd)

// if you need to remove rows:
// val sqlDelete = s"DELETE FROM $TargetTable WHERE experiment_name in ('Dunnings_letters_FR_AA_v4')  "
// dwhSend(sqlDelete)

sqlDelete: String = "DELETE FROM zps_analytics.ab_test_calculations_new_approach WHERE experiment_name in ('Test_of_new_feature_for_notebook_2','Dunning_Letters_AT','Dunnings_letters_FR_AA_v4') "

Insert SQL query in cmd9 that counts needed metrics grouped by customer_id and variant type. 
Use date parameters from config ('test_start_date' and 'day' as date between calc_start_date and calc_end_date):

In [0]:
df_results=pd.DataFrame()
  # create a list of dates for calculations
if len(dates)<3:
  dates=pd.date_range(calc_start_date,calc_end_date,freq=calc_frequency).strftime('%Y-%m-%d').tolist()
calculations_time=pd.to_datetime("today").strftime('%Y-%m-%d-%H-%M-%S')
print(dates)
for day in dates:

#  Actions needed!
#   insert your query with data grouped by customer_id and group type (control/treatment). Add date parameters for test_start_date and day of calculations.

  query= f"""
with test_variants as (
        select
            customer_number
            , min(payload_variant) as payload_variant
            , min(occurred_at) as assigned_at
        from ods.ods_f_octopus_feedback
        where payload_experiment_id in ('f1e6574c-a5a5-4f51-be73-c472e3bc91b9'
            , 'b9b3b401-da13-434f-b970-224ce3b9c01e')
         and payload_mode like 'TEST'
        group by 1)

select
      cu.customer_number,
      t.payload_variant,
      sum(gmv_bef_return) as gmv,
      sum(total_items) as items,
      count(distinct order_number) as orders,
  case when gmv>100 then 1 else 0 end as over100_customers, 1 as all_customers
  from dwh.v_sales_order o, dwh.v_sales_channel ch, dwh.v_customer cu, test_variants t
  where o.sk_sales_channel=ch.sk_sales_channel and cu.sk_customer=o.sk_customer and t.customer_number=cu.customer_number
    and sk_payment_method = 3 
    and cancelation_status != 'FULL'
    and country_shop = 'DE Shop'
    and order_date::date  between '"""+test_start_date+"""' and '"""+day+"""'
    and t.assigned_at >= '"""+test_start_date+"""' 
    and t.payload_variant is not Null
  group by 1,2
"""
  df_day_results = main_ab_test_calculation_function(day, calc_start_date, calc_end_date, metrics_columns, need_bootstrap, need_buckets, need_mde, proportions_metrics_flag, proportions_columns, ratio_metrics_flag, ratio_columns, query)
  df_results = df_results.append(df_day_results, ignore_index=False)

df_to_redshift = spark.createDataFrame(df_results)
df_to_redshift.createOrReplaceTempView("tmp")
df_results.head()


['2022-06-12', '2022-06-19', '2022-06-26', '2022-07-03', '2022-07-10', '2022-07-17']
2022-06-12

In [0]:
%scala
// change nothing here, just run it
var df_to_redshift_final = sql("""
select      
   experiment_name::string
   ,cast(dt as date) dt
   ,metric::string
   ,metric_type::string
   ,mean_control::double
   ,mean_treatment::double
   ,mean_diff::double
   ,mean_control_wo_outliers::double
   ,mean_treatment_wo_outliers::double
   ,mean_diff_wo_outliers::double
   ,median_control::double
   ,median_treatment::double
   ,median_diff::double
   ,ttest_pval::double
   ,ttest_mean_diff::double
   ,ttest_ci_prc_low::double
   ,ttest_ci_prc_up::double
   ,ttest_wo_outliers_pval::double
   ,ttest_wo_outliers_mean_diff::double
   ,ttest_wo_outliers_ci_prc_low::double
   ,ttest_wo_outliers_ci_prc_up::double
   ,btstrp_pval::double
   ,btstrp_mean_diff::double
   ,btstrp_ci_prc_low::double
   ,btstrp_ci_prc_up::double
   ,btstrp_wo_outliers_pval::double
   ,btstrp_wo_outliers_mean_diff::double
   ,btstrp_wo_outliers_ci_prc_low::double
   ,btstrp_wo_outliers_ci_prc_up::double
   ,buckets_pval::double
   ,buckets_mean_diff::double
   ,buckets_ci_prc_low::double
   ,buckets_ci_prc_up::double
   ,buckets_wo_outliers_pval::double
   ,buckets_wo_outliers_mean_diff::double
   ,buckets_wo_outliers_ci_prc_low::double
   ,buckets_wo_outliers_ci_prc_up::double
   ,MW_pval::double
   ,chi2_pval::double                        
   ,outliers_percentile::double
   ,outliers_threshold::double
   ,calculations_time::string
   ,mde_bootstrap::double
   ,mde_ttest_wo_outliers::double
   ,units_control::double
   ,units_treatment::double
   ,srm_pval::double
from tmp""")
display(df_to_redshift_final)

experiment_name,dt,metric,metric_type,mean_control,mean_treatment,mean_diff,mean_control_wo_outliers,mean_treatment_wo_outliers,mean_diff_wo_outliers,median_control,median_treatment,median_diff,ttest_pval,ttest_mean_diff,ttest_ci_prc_low,ttest_ci_prc_up,ttest_wo_outliers_pval,ttest_wo_outliers_mean_diff,ttest_wo_outliers_ci_prc_low,ttest_wo_outliers_ci_prc_up,btstrp_pval,btstrp_mean_diff,btstrp_ci_prc_low,btstrp_ci_prc_up,btstrp_wo_outliers_pval,btstrp_wo_outliers_mean_diff,btstrp_wo_outliers_ci_prc_low,btstrp_wo_outliers_ci_prc_up,buckets_pval,buckets_mean_diff,buckets_ci_prc_low,buckets_ci_prc_up,buckets_wo_outliers_pval,buckets_wo_outliers_mean_diff,buckets_wo_outliers_ci_prc_low,buckets_wo_outliers_ci_prc_up,MW_pval,chi2_pval,outliers_percentile,outliers_threshold,calculations_time,mde_bootstrap,mde_ttest_wo_outliers,units_control,units_treatment,srm_pval
Test_of_new_feature_for_notebook_2,2022-06-12,sold_items_aft_rett,continuous,1.6151,1.6189,0.0024,1.6145,1.6172,0.0017,1.0,1.0,0.0,0.6342,0.00235,-0.0073,0.01202,0.7309,0.00168,-0.00792,0.01125,0.61,0.00241,-0.00769,0.0124,0.774,0.00154,-0.00746,0.01135,null,null,null,null,null,null,null,null,0.35770951886333213,null,0.99999,73.86528000002727,2022-11-22-17-55-36,0.01,0.01,145976.0,146137.0,0.7658
Test_of_new_feature_for_notebook_2,2022-06-12,gmv_aft_ret_prov,continuous,55.2921,55.3394,9.0E-4,55.2654,55.2599,-1.0E-4,35.0,35.41,0.0117,0.8698,8.6E-4,-0.00945,0.011,0.9842,-1.0E-4,-0.01025,0.00973,0.93,5.3E-4,-0.00979,0.01087,0.934,-1.2E-4,-0.00977,0.01053,null,null,null,null,null,null,null,null,0.3580737327881923,null,0.99999,4067.992776601782,2022-11-22-17-55-36,0.03,0.03,145976.0,146137.0,0.7658
Test_of_new_feature_for_notebook_2,2022-06-12,num_orders/num_sessions,ratio,0.7987,0.8091,0.013,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.01316,0.00897,0.01753,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-11-22-17-55-36,0.005,null,145976.0,146137.0,0.7658
Test_of_new_feature_for_notebook_2,2022-06-19,sold_items_aft_rett,continuous,2.0191,2.0162,-0.0014,2.0184,2.0145,-0.0019,1.0,1.0,0.0,0.6133,-0.00142,-0.00685,0.0042,0.4888,-0.00192,-0.0073,0.00361,0.596,-0.0014,-0.00702,0.00447,0.494,-0.00193,-0.00744,0.00342,null,null,null,null,null,null,null,null,0.3029435442068812,null,0.99999,75.0,2022-11-22-17-55-36,0.01,0.01,402840.0,403215.0,0.6762
Test_of_new_feature_for_notebook_2,2022-06-19,gmv_aft_ret_prov,continuous,67.7947,67.7212,-0.0011,67.7603,67.6344,-0.0019,44.92999999999999,44.94938,4.0E-4,0.7182,-0.00109,-0.00703,0.00475,0.5221,-0.00186,-0.00767,0.00371,0.708,-0.00104,-0.00676,0.00468,0.488,-0.00203,-0.00772,0.0035,null,null,null,null,null,null,null,null,0.2592420203822586,null,0.99999,3488.5140265808577,2022-11-22-17-55-36,0.01,0.01,402840.0,403215.0,0.6762
Test_of_new_feature_for_notebook_2,2022-06-19,num_orders/num_sessions,ratio,0.8104,0.8182,0.0096,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.00968,0.00705,0.01232,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-11-22-17-55-36,0.005,null,402840.0,403215.0,0.6762
Test_of_new_feature_for_notebook_2,2022-06-26,sold_items_aft_rett,continuous,2.3189,2.3128,-0.0026,2.3177,2.3097,-0.0035,1.0,1.0,0.0,0.2661,-0.00263,-0.00717,0.00208,0.1299,-0.00343,-0.00791,9.4E-4,0.274,-0.00259,-0.00717,0.00214,0.148,-0.00339,-0.00778,0.00105,null,null,null,null,null,null,null,null,0.06797575568825086,null,0.99999,89.0,2022-11-22-17-55-36,0.005,0.005,605420.0,605678.0,0.8146
Test_of_new_feature_for_notebook_2,2022-06-26,gmv_aft_ret_prov,continuous,77.5327,77.4606,-9.0E-4,77.4748,77.2931,-0.0023,50.0,49.99,-2.0E-4,0.7248,-9.3E-4,-0.00606,0.00429,0.3218,-0.00235,-0.00701,0.00228,0.728,-9.3E-4,-0.00627,0.00428,0.35,-0.00228,-0.00679,0.0022,null,null,null,null,null,null,null,null,0.08370758177215654,null,0.99999,4353.561425600895,2022-11-22-17-55-36,0.005,0.005,605420.0,605678.0,0.8146
Test_of_new_feature_for_not

In [0]:
%scala
//  write tests results in Redshift TargetTable defined in cmd 6 
val sqlPreRun = s"select * from $TargetTable"
dwhWrite(df_to_redshift_final, TargetTable, sqlPreRun)

sqlPreRun: String = select * from zps_analytics.ab_test_calculations_new_approach